# Экспериментируем с генеалогическим деревом

Рассмотрим пример использования онтологии родственных отношений для поиска родственников в генеалогическом дереве. В качестве примера рассмотрим родословное дерево царской семьи, как наиболее доступное и подробное. Примеры вдохновлены [вот этой статьёй](https://habr.com/post/270857/).

Для работы с генеалогическими данными используется формат [GEDCOM](https://en.wikipedia.org/wiki/GEDCOM). Возьмём родословную, транслитерированную для простоты на латиницу:

In [1]:
!wget https://raw.githubusercontent.com/blokhin/genealogical-trees/master/data/tsars.ged
!head tsars.ged

--2018-11-16 20:36:44--  https://raw.githubusercontent.com/blokhin/genealogical-trees/master/data/tsars.ged
Resolving webproxy (webproxy)... 10.36.35.1
Connecting to webproxy (webproxy)|10.36.35.1|:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 4540 (4.4K) [text/plain]
Saving to: ‘tsars.ged.8’

tsars.ged.8         100%[===================>]   4.43K  --.-KB/s    in 0.002s  

2018-11-16 20:36:45 (2.49 MB/s) - ‘tsars.ged.8’ saved [4540/4540]

0 HEAD
1 CHAR UTF8
1 GEDC
2 VERS 5.5
0 @0@ INDI
1 NAME Mihail Fedorovich /Romanov/
1 SEX M


Попробуем вытащить полезную информацию из этого родословного дерева. Для начала установим библиотеку `python-gedcom` из репозитория GitHub:

In [2]:
import sys
!{sys.executable} -m pip install git+git://github.com/nickreynke/python-gedcom.git#egg=python-gedcom
!rm -rf src

Для работы с записями в файле создаём объект `Gedcom`. К сожалению, объект предоставляет лишь достаточно низкоуровневой доступ к файлу, позволяя по сути читать отдельные записи (об индивиддумах и семьях).

In [63]:
from gedcom import Gedcom
g = Gedcom('tsars.ged')

FileNotFoundError: [Errno 2] No such file or directory: 'tsars.ged'

In [4]:
d = g.get_element_dictionary()
[ (k,v.get_name()) for k,v in d.items()]

[('@0@', ('Mihail Fedorovich', 'Romanov')),
 ('@1@', ('Evdokija Lukjanovna', 'Streshneva')),
 ('@2@', ('Aleksej Mihajlovich', 'Romanov')),
 ('@3@', ('Marija Ilinichna', 'Miloslavskaja')),
 ('@4@', ('Natalja Kirillovna', 'Naryshkina')),
 ('@5@', ('Marfa Matveevna', 'Apraksina')),
 ('@6@', ('Fedor Alekseevich', 'Romanov')),
 ('@7@', ('Sofja Aleksevna', 'Romanova')),
 ('@8@', ('Ivan V Alekseevich', 'Romanov')),
 ('@9@', ('Praskovja Fedorovna', 'Saltykova')),
 ('@10@', ('Ekaterina Ivanovna', 'Romanova')),
 ('@11@', ('Anna Ivanovna', 'Romanova')),
 ('@12@', ('Fridrih Vilgelm', 'Kurlandskij')),
 ('@13@', ('Karl Leopold', 'Meklenburg-Shverinskij')),
 ('@14@', ('Anna Leopoldovna', 'Meklenburg-Shverinskaja')),
 ('@15@', ('Anton Ulrih', 'Braunshvejg-Volfenbjuttelskij')),
 ('@16@', ('Ivan VI Antonovich', 'Braunshvejg-Volfenbjuttelskij')),
 ('@17@', ('Petr I Alekseevich', 'Romanov')),
 ('@18@', ('Evdokija Fedorovna', 'Lopuhina')),
 ('@19@', ('Ekaterina I Alekseevna', 'Mihajlova')),
 ('@20@', ('Ale

Следующим шагом скачаем онтологию родства, описанную в формате троек Semantic Web. В этой онтологии описаны такие родственные отношения, как дядя (`isUncleOf`), кузен (`isCousinOf`) и множество других. При этом эти отношения описаны на основе базовых родственных отношений, и с помощью системы рассуждений мы должны быть в состоянии вывести все эти дополнительные отношения автоматически.

In [5]:
!wget https://raw.githubusercontent.com/blokhin/genealogical-trees/master/data/header.ttl
!mv header.ttl onto.ttl
!head -20 onto.ttl

--2018-11-16 20:36:56--  https://raw.githubusercontent.com/blokhin/genealogical-trees/master/data/header.ttl
Resolving webproxy (webproxy)... 10.36.35.1
Connecting to webproxy (webproxy)|10.36.35.1|:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 9220 (9.0K) [text/plain]
Saving to: ‘header.ttl’

header.ttl          100%[===================>]   9.00K  --.-KB/s    in 0.002s  

2018-11-16 20:36:56 (4.05 MB/s) - ‘header.ttl’ saved [9220/9220]

@prefix fhkb: <http://www.example.com/genealogy.owl#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://www.example.com/genealogy.owl#> a owl:Ontology .

fhkb:DomainEntity a owl:Class .

fhkb:Man a owl:Class ;
    owl:equivalentClass [ a owl:Class ;
            owl:intersectionOf ( fhkb:Person [ a

Для удобства получим единую онтологию, в которой к имеющимся правилам допишем в конец сведения о конкретных родственниках, полученные из файла GedCom. Для этого пройдем по всему файлу и вытащим информацию об индивиддумах и семьях, и преобразуем их в формат троек **turtle**.

In [6]:
gedcom_dict = g.get_element_dictionary()
individuals, marriages = {}, {}

def term2id(el):
    return "i" + el.get_pointer().replace('@', '').lower()

out = open("onto.ttl","a")

for k, v in gedcom_dict.items():
    if v.is_individual():
        children, siblings = set(), set()
        idx = term2id(v)

        title = v.get_name()[0] + " " + v.get_name()[1]
        title = title.replace('"', '').replace('[', '').replace(']', '').replace('(', '').replace(')', '').strip()

        own_families = g.get_families(v, 'FAMS')
        for fam in own_families:
            children |= set(term2id(i) for i in g.get_family_members(fam, "CHIL"))

        parent_families = g.get_families(v, 'FAMC')
        if len(parent_families):
            for member in g.get_family_members(parent_families[0], "CHIL"): # NB adoptive families i.e len(parent_families)>1 are not considered (TODO?)
                if member.get_pointer() == v.get_pointer():
                    continue
                siblings.add(term2id(member))

        if idx in individuals:
            children |= individuals[idx].get('children', set())
            siblings |= individuals[idx].get('siblings', set())
        individuals[idx] = {'sex': v.get_gender().lower(), 'children': children, 'siblings': siblings, 'title': title}

    elif v.is_family():
        wife, husb, children = None, None, set()
        children = set(term2id(i) for i in g.get_family_members(v, "CHIL"))

        try:
            wife = g.get_family_members(v, "WIFE")[0]
            wife = term2id(wife)
            if wife in individuals: individuals[wife]['children'] |= children
            else: individuals[wife] = {'children': children}
        except IndexError: pass
        try:
            husb = g.get_family_members(v, "HUSB")[0]
            husb = term2id(husb)
            if husb in individuals: individuals[husb]['children'] |= children
            else: individuals[husb] = {'children': children}
        except IndexError: pass

        if wife and husb: marriages[wife + husb] = (term2id(v), wife, husb)

for idx, val in individuals.items():
    added_terms = ''
    if val['sex'] == 'f':
        parent_predicate, sibl_predicate = "isMotherOf", "isSisterOf"
    else:
        parent_predicate, sibl_predicate = "isFatherOf", "isBrotherOf"
    if len(val['children']):
        added_terms += " ;\n    fhkb:" + parent_predicate + " " + ", ".join(["fhkb:" + i for i in val['children']])
    if len(val['siblings']):
        added_terms += " ;\n    fhkb:" + sibl_predicate + " " + ", ".join(["fhkb:" + i for i in val['siblings']])
    out.write("fhkb:%s a owl:NamedIndividual, owl:Thing%s ;\n    rdfs:label \"%s\" .\n" % (idx, added_terms, val['title']))

for k, v in marriages.items():
    out.write("fhkb:%s a owl:NamedIndividual, owl:Thing ;\n    fhkb:hasFemalePartner fhkb:%s ;\n    fhkb:hasMalePartner fhkb:%s .\n" % v)

out.write("[] a owl:AllDifferent ;\n    owl:distinctMembers (")
for idx in individuals.keys():
    out.write("    fhkb:" + idx)
for k, v in marriages.items():
    out.write("    fhkb:" + v[0])
out.write("    ) .")
out.close()

In [62]:
!tail onto.ttl

tail: cannot open 'onto.ttl' for reading: No such file or directory


Следующим шагом нам потребуется работать с получившейся онтологией семейства. Для этого используем библиотеку [RDFLib](https://github.com/RDFLib), которая позволяет читать RDF Graph в различных форматах, формулировать к нему запросы и т.д. 

Для проведения логического вывода используем библиотеку [RDFClosure](https://github.com/RDFLib/OWL-RL), которая позволит нам получить **замыкание** графа, т.е. добавить в него все возможные выводимые конструкции, что впоследствии сделать к ним запрос.

In [15]:
!{sys.executable} -m pip install rdflib
!{sys.executable} -m pip install git+git://github.com/RDFLib/OWL-RL.git#egg=RDFClosure

  Cloning git://github.com/RDFLib/OWL-RL.git to /tmp/pip-install-3jouot5s/RDFClosure
  Running setup.py bdist_wheel for RDFClosure ... done
  Stored in directory: /tmp/pip-ephem-wheel-cache-13in8fda/wheels/d8/db/e8/a1d3dea0a6029d7e76b153cfb7129a343a1e357c055c87896b
Successfully built RDFClosure


Открываем файл с онтологией и смотрим, сколько триплетов в нем изначально находится.

In [51]:
import rdflib
from RDFClosure import DeductiveClosure, OWLRL_Extension

g = rdflib.Graph()
g.parse("onto.ttl", format="turtle")

print("Triplets found:%d" % len(g))

Triplets found:669


Теперь осуществляем логический вывод и смотрим на количество получившихся триплетов:

In [53]:
DeductiveClosure(OWLRL_Extension).expand(g)
print("Triplets after inference:%d" % len(g))

Triplets after inference:4202


Запрос к графу удобнее всего делать на языке **SPARQL**. В нашем случае мы определим, кто является чьим дядей:

In [56]:
qres = g.query(
    """SELECT DISTINCT ?aname ?bname
       WHERE {
          ?a fhkb:isUncleOf ?b .
          ?a rdfs:label ?aname .
          ?b rdfs:label ?bname .
       }""")

for row in qres:
    print("%s is uncle of %s" % row)

Aleksandr I Pavlovich Romanov is uncle of Aleksandr II Nikolaevich Romanov
Fedor Alekseevich Romanov is uncle of Anna Ivanovna Romanova
Fedor Alekseevich Romanov is uncle of Ekaterina Ivanovna Romanova


В заключение удалим все лишние файлы

In [60]:
!rm tsars.ged onto.ttl

rm: cannot remove 'tsars.ged': No such file or directory
